<div class="alert alert-block alert-success">

<!-- -info-warning-success-danger>   --> <font size=5>
<b>
    СВЕРКА ОБЛАКА и 1С<br>
        </b>    </div>
<hr>

In [1]:
# pd.set_option('display.expand_frame_repr', True)
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', None)

In [2]:
import pandas as pd
import numpy as np
import warnings
from IPython.display import display_markdown
warnings.filterwarnings('ignore')

####
<div class="alert alert-block alert-success"> <font size=5>
<b>подключаем облако и 605й (1c)
    </b>    </div>
<hr>

In [3]:
file_cloud = 'Реестр_605_новая_форма_24112023_0833.csv'
df_cloud = pd.read_csv(file_cloud, delimiter=';', encoding='cp1251')



file = 'выгрузка_из_1с_2411.csv'
df_1c = pd.read_csv(file, delimiter=';', encoding='cp1251')


print(df_cloud.shape)
print(df_1c.shape)

(13335, 32)
(109185, 39)


<div class="alert alert-block alert-info"> <!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">     -->
<font size=3>
<b>
    Убираем лишние колонки для 605го и переименовываем
    </b>
    </div>
<hr>

In [4]:
# как4ие есть колонки
# df_cloud.columns.to_list()

In [5]:
# как4ие есть колонки
df_1c.columns.to_list()

['ID  пациента',
 'Дата включения в реестр пациентов, нуждающихся в ПМП',
 'Дата постановки на учет/включения пациента в реестр',
 'Пациент',
 'Пол',
 'Дата рождения',
 'Гражданский статус',
 'Адрес',
 'Адрес фактический',
 'Документ удостоверяющий личность',
 'Серия/№',
 'Кем выдан',
 'Свидетельство о регистрации по месту жительства(до 14 лет)',
 'СНИЛС',
 'Полис ОМС',
 'Наличие инвалидности',
 'Основной диагноз по МКБ-10',
 'Основное контактное лицо',
 'Дата признания пациента нуждающимся в ПМП (ВК)',
 'Тип документа о признании паллиативным',
 'Медицинская организация, в которой признали пациента нуждающимся в ПМП',
 'Наличие разрешения ДЗМ на оказание помощи',
 'Дата разрешения ДЗМ',
 'Номер разрешения ДЗМ',
 'Текущий статус пациента',
 'Дата прекращения оказания ПМП',
 'Основания прекращения оказания пациенту ПМП',
 'Курирующее подразделение/филиал',
 'Наблюдение в РЦ',
 'БК',
 'Основной диагноз на момент обращения',
 'Комментарий',
 'Участник СВО',
 'Канал поступления информации'

<div class="alert alert-block alert-info"> <font size=3>
<b> Готовим ключи для женитьбы</b><br>
        </div>
<hr>

In [6]:
# ----------- Ё --------
df_1c['Пациент'] = df_1c['Пациент'].str.replace('ё', 'е') 
df_cloud['ФИО пациента'] = df_cloud['ФИО пациента'].str.replace('ё', 'е') 

<div class="alert alert-block alert-info">
<font size=1>
<b>                                   Приведем ДАТЫ к фрмату ДАТА</b><br>
            </div>

In [7]:
# ----------------форматнем дату к дате
df_1c['Дата рождения'] = pd.to_datetime(df_1c['Дата рождения'], dayfirst=True, errors = 'coerce').dt.normalize()
df_cloud['Дата рождения'] = pd.to_datetime(df_cloud['Дата рождения'], dayfirst=True, errors = 'coerce').dt.normalize()

In [8]:
#  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------
df_1c['Ключ'] = df_1c['Пациент'].map(str) + (df_1c['Дата рождения'].dt.date).map(str)
df_1c['Ключ'] = df_1c['Ключ'].str.lower()            # ---- все к нижнему регистру
df_1c['Ключ'] = df_1c['Ключ'].str.replace(' ', '')   # ---- пробелы долой 
print(df_1c.shape)

#  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------
df_cloud['Ключ'] = df_cloud['ФИО пациента'].map(str) + (df_cloud['Дата рождения'].dt.date).map(str)
df_cloud['Ключ'] = df_cloud['Ключ'].str.lower()            # ---- все к нижнему регистру
df_cloud['Ключ'] = df_cloud['Ключ'].str.replace(' ', '')   # ---- пробелы долой 
print(df_cloud.shape)

(109185, 40)
(13335, 33)


<div class="alert alert-block alert-info">
<font size=1>
<b>  УБИРАЕМ СТАТУСЫ КРИВЫЕ</b><br>
            </div>

In [9]:
df_1c = df_1c.loc[df_1c['Пометка удаления'] != 'Да']
df_1c.shape

(109146, 40)

In [10]:
list_key = ['неизвестноnat','непредставилсяnat','нетnat','неизвестныйnat']
df_1c = df_1c.loc[~df_1c['Ключ'].isin(list_key)]
df_1c = df_1c.drop_duplicates('Ключ', keep='last')
df_1c.shape

(109072, 40)

In [11]:
# df_1c[df_1c.duplicated('Код', keep=False)].sort_values('Пациент')
# df.columns = df.columns.str.rstrip('_x')

<div class="alert alert-block alert-info"> <font size=3>
<b> Джойним датасет ОБЛАКО с 605м<br>
</b>            </div>

In [12]:
df_cloud = df_cloud.add_suffix('_облако')
df_cloud.rename(columns={'Ключ_облако': 'Ключ'}, inplace=True)
df_1c = df_1c.add_suffix('_1с')
df_1c.rename(columns={'Ключ_1с': 'Ключ'}, inplace=True)

In [13]:
df_merg = pd.merge(df_cloud, df_1c, how='left', on='Ключ')
df_merg.shape

(13335, 72)

###
<div class="alert alert-block alert-success"> <font size=3>
<b> Проверка пациента, НА ОПЕЧАТКИ (есть в облаке - нет в 1С)<br>
</b>            </div>

In [14]:
df_opechatka_fio_dr = df_merg.loc[df_merg['Пациент_1с'].isna()]
df_opechatka_fio_dr.dropna(how='all', axis=1, inplace=True)
df_opechatka_fio_dr.to_excel('df_opechatka_fio_dr.xlsx')
df_merg = df_merg.loc[~df_merg['Пациент_1с'].isna()]

In [15]:
df_opechatka_fio_dr.shape
df_opechatka_fio_dr.to_excel('df_opechatka_fio_dr.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Дубли ОБЛАКА - ФИО+ДР <br>
</b>            </div>

In [16]:
duble_cloude_fio = df_merg[df_merg.duplicated('Ключ', keep=False)].sort_values('Ключ')

In [17]:
duble_cloude_fio = duble_cloude_fio[[
    'Дата включения пациента в Реестр_облако',
     'ФИО пациента_облако',
     'Пол_облако',
     'Дата рождения_облако',
     'Гражданский статус_облако',
    'СНИЛС_облако',
    'СНИЛС_1с',
    'Пациент_1с'
]]
duble_cloude_fio.shape
duble_cloude_fio.to_excel('duble_cloude_fio.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Дубли ОБЛАКА - CНИЛС <br>
</b>            </div>

In [18]:
# --------- преобразовать написание СНИЛС -----
df_dub_snils = df_merg.copy()
df_dub_snils

df_dub_snils['СНИЛС_облако'] = df_dub_snils['СНИЛС_облако'].str.replace('-', '')
df_dub_snils['СНИЛС_облако'] = df_dub_snils['СНИЛС_облако'].str.replace(' ', '')
df_dub_snils['СНИЛС_облако'] = df_dub_snils['СНИЛС_облако'].str.replace('/', '')
df_dub_snils['СНИЛС_облако'] = df_dub_snils['СНИЛС_облако'].str.replace('.', '')

In [19]:
df_dub_snils = df_dub_snils[df_dub_snils.duplicated('СНИЛС_облако', keep=False)]

df_dub_snils = df_dub_snils.loc[
    ~df_dub_snils['СНИЛС_облако'].isna()][[
    'Дата включения пациента в Реестр_облако',
 'ФИО пациента_облако',
 'Пол_облако',
 'Дата рождения_облако',
 'Гражданский статус_облако',

 'СНИЛС_облако',
    'СНИЛС_1с',
    'Пациент_1с'
]]

In [20]:
df_dub_snils.to_excel('df_dub_snils.xlsx')
df_dub_snils.shape

(11, 8)

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Дубли ОБЛАКА - ПОЛИС <br>
</b>            </div>

In [21]:
# --------- преобразовать написание СНИЛС -----
df_dub_oms = df_merg.copy()
df_dub_oms

df_dub_oms['Полис ОМС_облако'] = df_dub_oms['Полис ОМС_облако'].str.replace('-', '')
df_dub_oms['Полис ОМС_облако'] = df_dub_oms['Полис ОМС_облако'].str.replace(' ', '')
df_dub_oms['Полис ОМС_облако'] = df_dub_oms['Полис ОМС_облако'].str.replace('/', '')
df_dub_oms['Полис ОМС_облако'] = df_dub_oms['Полис ОМС_облако'].str.replace('.', '')

In [22]:
df_dub_oms = df_dub_oms[df_dub_oms.duplicated('Полис ОМС_облако', keep=False)]

df_dub_oms = df_dub_oms.loc[
    ~df_dub_oms['Полис ОМС_облако'].isna()][[
    'Дата включения пациента в Реестр_облако',
 'ФИО пациента_облако',
 'Пол_облако',
 'Дата рождения_облако',
 'Гражданский статус_облако',

 'Полис ОМС_облако',
    'Полис ОМС_1с',
    'Пациент_1с'
]]

In [23]:
df_dub_oms.shape
df_dub_oms.to_excel('df_dub_oms.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Пустоты (облака) ДР<br>
</b>            </div>

In [24]:
df_cloud_pusto_dr = df_merg.loc[
       df_merg['Дата рождения_облако'].isna()
                                                                          ][[
    'id пациента_облако',
 'ФИО пациента_облако',
 'Дата рождения_облако',
    # 'Дата рождения_1с',
     'Дата включения пациента в Реестр_облако',
      'Дата постановки на учет/включения пациента в реестр_1с',
 'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
      'Текущий статус пациента_облако',   
 'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',
]]
df_cloud_pusto_dr.shape
df_cloud_pusto_dr.to_excel('df_cloud_pusto_dr.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Пустоты (облака) id-ЕМИАС<br>
</b>            </div>

In [25]:
df_merg['Дата включения пациента в Реестр_облако'] = pd.to_datetime(df_merg['Дата включения пациента в Реестр_облако'])

In [26]:
df_cloud_pusto_id_emias = df_merg.loc[
       df_merg['id пациента_облако'].isna()
    & (df_merg['Дата включения пациента в Реестр_облако'] > '2023-11-15')
                                                                          ][[
    'id пациента_облако',
 'ФИО пациента_облако',
 'Дата рождения_облако',
     'Дата включения пациента в Реестр_облако',
      'Дата постановки на учет/включения пациента в реестр_1с',
 'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
      'Текущий статус пациента_облако',   
 'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',
]]
df_cloud_pusto_id_emias.shape
df_cloud_pusto_id_emias.to_excel('df_cloud_pusto_id_emias.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Пустоты (облака) СНИЛС<br>
</b>            </div>

In [27]:
df_cloud_pusto_snils = df_merg.loc[
       df_merg['СНИЛС_облако'].isna()
                                      ][[
    'id пациента_облако',
    'ФИО пациента_облако',
    'Дата рождения_облако',
    'Дата включения пациента в Реестр_облако',
    'Дата постановки на учет/включения пациента в реестр_1с',
    'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
    'Текущий статус пациента_облако',   
    'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',
     'СНИЛС_1с',
]]
df_cloud_pusto_snils.shape
df_cloud_pusto_snils.to_excel('df_cloud_pusto_snils.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Пустоты (облака) ПОЛИС<br>
</b>            </div>

In [28]:
df_cloud_pusto_oms = df_merg.loc[
       df_merg['Полис ОМС_облако'].isna()
                                                                          ][[
    'id пациента_облако',
 'ФИО пациента_облако',
 'Дата рождения_облако',
     'Дата включения пациента в Реестр_облако',
      'Дата постановки на учет/включения пациента в реестр_1с',
 'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
      'Текущий статус пациента_облако',   
 'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',

    'Полис ОМС_1с',
]]
df_cloud_pusto_oms.to_excel('df_cloud_pusto_oms.xlsx')
print(df_cloud_pusto_oms.shape)

(1, 10)


###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Пустоты (облака) ПАСПОРТ<br>
</b>            </div>

In [29]:
df_cloud_pusto_pasp = df_merg.loc[
       df_merg['Номер документа, удостоверяющего личность_облако'].isna()
                                                                          ][[
    'id пациента_облако',
 'ФИО пациента_облако',
 'Дата рождения_облако',
     'Дата включения пациента в Реестр_облако',
      'Дата постановки на учет/включения пациента в реестр_1с',
 'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
      'Текущий статус пациента_облако',   
 'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',

    'Серия/№_1с',
]]
# df_cloud_pusto_pasp.to_excel('df_cloud_pusto_pasp.xlsx')
print(df_cloud_pusto_pasp.shape)

(1997, 10)


###
<div class="alert alert-block alert-success"> <font size=3>
<b>  пустая МО при наличии даты ВК (пациент - не новый)<br>
</b>            </div>

In [30]:
df_cloud_pusto_mo = df_merg.loc[
       ~df_merg['Дата выдачи заключения _облако'].isna()
    & (df_merg['Наименование медицинской организации, выдавшей медицинское заключение _облако'].isna()    )
      & (df_merg['Текущий статус пациента_облако'].str.contains('пациен') == False )
    
                                                                          ][[
    'id пациента_облако',
    'ФИО пациента_облако',
    'Дата рождения_облако',

    'Дата включения пациента в Реестр_облако',
    'Дата выдачи заключения _облако',
    'Наименование медицинской организации, выдавшей медицинское заключение _облако',
    'Медицинская организация, в которой признали пациента нуждающимся в ПМП_1с',
    
    'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
    'Текущий статус пациента_облако',   
    'Курирующее подразделение/филиал_1с',
    'Текущий статус пациента_1с',
    ]]
df_cloud_pusto_mo.shape
df_cloud_pusto_mo.to_excel('df_cloud_pusto_mo.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Пустоты (облака) дата ВК (статус "не новый")<br>
</b>            </div>

In [31]:
df_cloud_pusto_vk = df_merg.loc[
       df_merg['Дата выдачи заключения _облако'].isna()
    & (df_merg['Текущий статус пациента_облако'].str.contains('пациен') == False )
    
                                                                          ][[
    'id пациента_облако',
 'ФИО пациента_облако',
 'Дата рождения_облако',
    'Дата выдачи заключения _облако',
     'Дата включения пациента в Реестр_облако',
      'Дата постановки на учет/включения пациента в реестр_1с',
 'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
      'Текущий статус пациента_облако',   
 'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',
]]
df_cloud_pusto_vk.shape
df_cloud_pusto_vk.to_excel('df_cloud_pusto_vk.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Пустая Дата разрешения (облако) если Гражд статус "немосквич" <br>
</b>            </div>

In [32]:
df_cloud_pusto_razresh = df_merg.loc[
       df_merg['Дата выдачи разрешения_облако'].isna()
    & (df_merg['Гражданский статус_облако'].str.contains('осквич') == False )
    
                                                                          ][[
    'id пациента_облако',
 'ФИО пациента_облако',
 'Дата рождения_облако',
    'Дата выдачи разрешения_облако',
     'Дата включения пациента в Реестр_облако',
      'Гражданский статус_облако',
 'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
      'Текущий статус пациента_облако',   
 'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',
]]
df_cloud_pusto_razresh.shape
df_cloud_pusto_razresh.to_excel('df_cloud_pusto_razresh.xlsx')

### 
<hr>

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Дата включения (облако) <> Дата вкл в гор.реестр<br>
</b>            </div>

In [33]:
# df_merg.shape

In [34]:
# df_differ_date_includs_reestr = df_merg.loc[
#        ~df_merg['Дата включения пациента в Реестр_облако'].isna()
#     & (df_merg['Дата включения пациента в Реестр_облако'] != df_merg['Дата включения в реестр пациентов, нуждающихся в ПМП_1с'])
#     & (~df_merg['Текущий статус пациента_облако'].isin(['Новый пациент','новый пациент']))
#                                                                        ][[
#  'ФИО пациента_облако',
#  'Дата рождения_облако',
#      'Дата включения пациента в Реестр_облако',
#       'Дата постановки на учет/включения пациента в реестр_1с',
#     'Дата включения в реестр пациентов, нуждающихся в ПМП_1с',
#  'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
#       'Текущий статус пациента_облако',   
#  'Курирующее подразделение/филиал_1с',
#      'Текущий статус пациента_1с',
# ]]
# df_differ_date_includs_reestr.shape
# df_differ_date_includs_reestr

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  'Текущий статус пациентас_x', - 'Текущий статус пациента_y',<br>
</b>            </div>

In [35]:
# df_merg['Текущий статус пациента_облако'] = df_merg['Текущий статус пациента_облако'].str.lower()
# df_merg['Текущий статус пациента_1с'] = df_merg['Текущий статус пациента_1с'].str.lower()


In [36]:
# df_differ_now_status = df_merg.loc[
      
#     ((df_merg['Текущий статус пациента_облако'].str.contains('пациент') == False) &  (df_merg['Текущий статус пациента_1с'].str.contains('пациент') == False))
#     &
#     (
#         df_merg['Текущий статус пациента_облако'].str.contains('стационар') & (((df_merg['Текущий статус пациента_1с'] != ''.str.contains('стационар') == False) | (df_merg['Текущий статус пациента_1с'].str.contains('госпит') == False  )))
#     )
    
    
# #     & (
# #              ((df_merg['Текущий статус пациента_облако'] == 'прекращено оказание пмп') & (df_merg['Причина прекращения ПМП_облако'] == 'смерть пациента')) 
# #             & (df_merg['Текущий статус пациента_1с'] != 'умер')
# #       )   
    
       
#     # & (df_merg['Текущий статус пациента_облако'] != df_merg['Текущий статус пациента_1с'])
    

# ][[
#  'ФИО пациента_облако',
#  'Дата рождения_облако',
#  'Дата включения пациента в Реестр_облако',
#  'Дата постановки на учет/включения пациента в реестр_1с',
#  'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
      
#  'Курирующее подразделение/филиал_1с',
#  'Текущий статус пациента_облако',   
#     'Причина прекращения ПМП_облако',
#  'Текущий статус пациента_1с',
#     'Клиент.Дата прекращения оказания ПМП_облако',
#     'Дата прекращения оказания ПМП_1с',
# ]]
# df_differ_now_status.shape

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  'Пустой диагноз' облако  (есть в 1с),<br>
</b>            </div>

In [37]:
# df_merg['Текущий статус пациента_облако'] = df_merg['Текущий статус пациента_облако'].str.lower()
# df_merg['Текущий статус пациента_1с'] = df_merg['Текущий статус пациента_1с'].str.lower()


In [38]:
df_cloud_pusto_diagnoz = df_merg.loc[
      
    # (~df_merg['Текущий статус пациента_облако'].str.contains('пациент') &  ~df_merg['Текущий статус пациента_1с'].str.contains('пациент'))
    df_merg['Основной диагноз_облако'].isna()
    & ~df_merg['Основной диагноз по МКБ-10_1с'].isna()
                                                                   ][[
 'ФИО пациента_облако',
 'Дата рождения_облако',
 'Дата включения пациента в Реестр_облако',
 'Дата постановки на учет/включения пациента в реестр_1с',
 'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
      
 'Курирующее подразделение/филиал_1с',
 'Текущий статус пациента_облако',   
 'Текущий статус пациента_1с',
    'Клиент.Дата прекращения оказания ПМП_облако',
    'Дата прекращения оказания ПМП_1с',
    'Основной диагноз_облако',
    'Основной диагноз по МКБ-10_1с',
    
]]
df_cloud_pusto_diagnoz.shape
df_cloud_pusto_diagnoz.to_excel('df_cloud_pusto_diagnoz.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b>  Отличия диагнозов между ОБЛАКОМ и 1С !!<br>
</b>            </div>

In [39]:
df_merg1 = df_merg.copy()

In [40]:
# df_merg['Текущий статус пациента_облако'] = df_merg['Текущий статус пациента_облако'].str.lower()
# df_merg['Текущий статус пациента_1с'] = df_merg['Текущий статус пациента_1с'].str.lower()
df_merg['Основной диагноз_облако'] = df_merg['Основной диагноз_облако'].astype(str)
df_merg['Основной диагноз по МКБ-10_1с'] = df_merg['Основной диагноз по МКБ-10_1с'].astype(str)

In [41]:
def digit_diagnoz(diagnoz_str):
    digit_ = ''.join(filter(str.isdigit, diagnoz_str))
   
    return f'{digit_}'

In [42]:
df_merg['digit_diagnoz_cloud'] = df_merg['Основной диагноз_облако'].apply(digit_diagnoz)
df_merg['digit_diagnoz_1c'] = df_merg['Основной диагноз по МКБ-10_1с'].apply(digit_diagnoz)

In [43]:
df_diff_diagnoz = df_merg.loc[
      
      ((df_merg['Основной диагноз_облако'] != 'nan') & (df_merg['Основной диагноз по МКБ-10_1с'] != 'nan'))
    
     & (df_merg['digit_diagnoz_cloud'] != df_merg['digit_diagnoz_1c'])
                                                                   ][[
 'ФИО пациента_облако',
 'Дата рождения_облако',
 'Дата включения пациента в Реестр_облако',
 'Дата постановки на учет/включения пациента в реестр_1с',
 'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
      
 'Курирующее подразделение/филиал_1с',
 'Текущий статус пациента_облако',   
 'Текущий статус пациента_1с',
    'Клиент.Дата прекращения оказания ПМП_облако',
    'Дата прекращения оказания ПМП_1с',
    'Основной диагноз_облако',
    'Основной диагноз по МКБ-10_1с',
    'digit_diagnoz_cloud', 'digit_diagnoz_1c',
]]
df_diff_diagnoz.shape

(200, 14)

In [44]:
df_diff_diagnoz.to_excel('df_diff_diagnoz.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b> статус заявки "новая" продолжительность более 5 дней
</b>            </div>

In [45]:
now = pd.to_datetime('today').normalize()

In [46]:
df_merg['Дата создания заявки_облако'] = pd.to_datetime(df_merg['Дата создания заявки_облако'])
df_merg['Дата создания заявки_облако'] = pd.to_datetime(df_merg['Дата создания заявки_облако'])

In [47]:
df_cloud_long_zayavka = df_merg.loc[
    (df_merg[ 'Статус заявки_облако'].str.contains('овая') == False)
    & ((now - df_merg['Дата создания заявки_облако']) > '5 days')                                                             
                                                                   ][[
                                                              
    'id пациента_облако',
 'ФИО пациента_облако',
 'Дата рождения_облако',
         'Дата включения пациента в Реестр_облако',
   'Статус заявки_облако',
      'Гражданский статус_облако',
 'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
      'Текущий статус пациента_облако',   
 'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',
]]
df_cloud_long_zayavka.shape

(50, 10)

In [48]:
df_cloud_long_zayavka.to_excel('df_cloud_long_zayavka.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b> статус заявки "новая", текущ статус - "не новый"
</b>            </div>

In [49]:
df_cloud_status_not_new = df_merg.loc[
    (df_merg[ 'Статус заявки_облако'].str.contains('овая') == True)
    & (df_merg['Текущий статус пациента_облако'].str.contains('овый') == False ) 
                                                                   ][[

 'ФИО пациента_облако',
 'Дата рождения_облако',
         'Дата включения пациента в Реестр_облако',
   'Статус заявки_облако',
    'Текущий статус пациента_облако',  
       'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
       
 'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',
]]
df_cloud_status_not_new.shape

(8, 8)

In [50]:
df_cloud_status_not_new.to_excel('df_cloud_status_not_new.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b> прекращено оказание - - причина Пустая
</b>            </div>

In [51]:
df_cloud_stop_pusto = df_merg.loc[
    (df_merg[ 'Текущий статус пациента_облако'].str.contains('оказание') == True)
    & (df_merg['Причина прекращения ПМП_облако'].isna()) 
                                                                   ][[

 'ФИО пациента_облако',
 'Дата рождения_облако',
         'Дата включения пациента в Реестр_облако',
    'Текущий статус пациента_облако',  
       'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
       
 'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',
]]
df_cloud_stop_pusto.shape

(4, 7)

In [52]:
df_cloud_stop_pusto.to_excel('df_cloud_stop_pusto.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b> прекращено оказание -- дата Пустая
</b>            </div>

In [53]:
df_cloud_prekrash_data_pusto = df_merg.loc[
    (df_merg[ 'Текущий статус пациента_облако'].str.contains('оказание') == True)
    & (df_merg['Клиент.Дата прекращения оказания ПМП_облако'].isna()) 
                                                                   ][[

 'ФИО пациента_облако',
 'Дата рождения_облако',
         'Дата включения пациента в Реестр_облако',
    'Текущий статус пациента_облако',  
    'Клиент.Дата прекращения оказания ПМП_облако',
       'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
       
 'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',
]]
df_cloud_prekrash_data_pusto.shape

(23, 8)

In [54]:
df_cloud_prekrash_data_pusto.to_excel('df_cloud_prekrash_data_pusto.xlsx')

###
<div class="alert alert-block alert-success"> <font size=3>
<b> причина - смерть - дата прекр Пустая
</b>            </div>

In [55]:
df_merg['Дата рождения_облако'] = pd.to_datetime(df_merg['Дата рождения_облако']).dt.date


In [56]:
df_cloud_dead_data_pusto = df_merg.loc[
    (df_merg[ 'Текущий статус пациента_облако'].str.contains('оказание') == True)
    &(df_merg[ 'Причина прекращения ПМП_облако'].str.contains('мерть') == True)
    & (
        (df_merg['Клиент.Дата прекращения оказания ПМП_облако'].isna()
       | (df_merg['Дата смерти_облако'].isna())       
        )) 
                                                                   ][[

 'ФИО пациента_облако',
 'Дата рождения_облако',
         'Дата включения пациента в Реестр_облако',
    'Текущий статус пациента_облако',  
    'Причина прекращения ПМП_облако',
    'Клиент.Дата прекращения оказания ПМП_облако',
    'Дата смерти_облако',
       'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако',
       
 'Курирующее подразделение/филиал_1с',
     'Текущий статус пациента_1с',
]]
df_cloud_dead_data_pusto

,ФИО пациента_облако,Дата рождения_облако,Дата включения пациента в Реестр_облако,Текущий статус пациента_облако,Причина прекращения ПМП_облако,Клиент.Дата прекращения оказания ПМП_облако,Дата смерти_облако,"Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_облако",Курирующее подразделение/филиал_1с,Текущий статус пациента_1с
521,Еремко Андрей Владимирович,1960-07-20,2023-06-29,прекращено оказание ПМП,смерть пациента,NaN,NaN,Люблино ОВПП,Люблино ОВПП,Умер
5335,Маренов Михаил Васильевич,1947-11-18,2023-10-14,прекращено оказание ПМП,смерть пациента,17.11.2023,NaN,Преображенское ОВПП,Преображенское ОВПП,Умер


###
<div class="alert alert-block alert-success"> <font size=3>
<b> Вывод

In [59]:
# ---------------------------- ИТОГИ ---------------
start = '24-11-2023_08:33'

god_start = start.split('-')[0]
mes_start = start.split('-')[1]
day_start = start.split('-')[2]

# ------------------------------------------------готовим реестр ------
df_opechatka_fio_dr.shape

# ------------------------------------------------собираем данные ------
# all_clients  = def_data_all_clients(df_reest, start, finish)
# new_clients  = def_data_new_clients(df_reest, start, finish)
# dead_clients = def_data_dead_clients(df_reest, start, finish)
# inogorod_clients = def_data_inogorod_clients(df_reest, start, finish)
# data_inogorod_clients_all = def_data_inogorod_clients_all(df_reest, start, finish)

# df_live = def_live_clients_today(df_reest)

a = f'Динамика мониторинга чистоты на {start} :'

display_markdown(f''' # {a}

| Параметр | Количество | Сохранен в файл |- | - |
|-:|:|:-|:-|:-|
| - __Всего__ записей в облаке| __{df_cloud.shape[0]}__      | - | - |- |
| - возможные __опечатки__| {df_opechatka_fio_dr.shape[0]}                      | df_opechatka_fio_dr.xlsx | - |- |
| - __Дубли__ ФИО+ДР (с повторами)             | {duble_cloude_fio.shape[0]}          | duble_cloude_fio.xlsx |---- |---- |
| - __Дубли__ СНИЛС (с повторами)              | {df_dub_snils.shape[0]}             | df_dub_snils.xlsx |---- |---- |
| - __Дубли__ Полис-ОМС (с повторами)          | {df_dub_oms.shape[0]}                  | df_dub_oms.xlsx |---- |---- |
| - пустой id-ЕМИАС (в облаке) (позже 15.11.23) | {df_cloud_pusto_id_emias.shape[0]}   | df_cloud_pusto_id_emias.xlsx |---- |---- |
| - пустой СНИЛС   (в облаке)              | {df_cloud_pusto_snils.shape[0]}   | df_cloud_pusto_snils.xlsx |---- |---- |
| - пустой ОМС  (в облаке)                 | {df_cloud_pusto_oms.shape[0]}   | df_cloud_pusto_oms.xlsx |---- |---- |
| - пустое ДР (в облаке)                 | {df_cloud_pusto_dr.shape[0]}   | df_cloud_pusto_dr.xlsx |---- |---- |
| - пустая дата ВК(в облаке)  (статус "не новый")              | {df_cloud_pusto_vk.shape[0]}   | df_cloud_pusto_vk.xlsx |---- |---- |
| - пустая МО при наличии даты ВК (пациент - не новый)               | {df_cloud_pusto_mo.shape[0]} |  df_cloud_pusto_mo.xlsx |---- |---- |
| - Пустая Дата разрешения (облако) если Гражд статус "не москвич"               | {df_cloud_pusto_razresh.shape[0]} |  df_cloud_pusto_razresh.xlsx |---- |---- |
| - Пустой диагноз в облаке  (есть в 1с)   | {df_cloud_pusto_diagnoz.shape[0]} |  df_cloud_pusto_diagnoz.xlsx |---- |---- |
| - статус заявки "новая" продолжительность более 5 дней                | {df_cloud_long_zayavka.shape[0]}  | df_cloud_long_zayavka.xlsx |---- |---- |
| - статус заявки "новая", текущ статус - "не новый"                | {df_cloud_status_not_new.shape[0]}  | df_cloud_status_not_new.xlsx |---- |---- |
| - прекращено оказание (в облаке) причина Пустая                | {df_cloud_pusto_dr.shape[0]}  | df_cloud_pusto_dr.xlsx |---- |---- |
| - прекращено оказание (в облаке) дата Пустая                | {df_cloud_prekrash_data_pusto.shape[0]}   |df_cloud_prekrash_data_pusto.xlsx |---- |---- |
| - причина - смерть - дата прекр Пустая                | {df_cloud_dead_data_pusto.shape[0]}   |df_cloud_dead_data_pusto.xlsx |---- |---- |
| - Отличия диагнозов между ОБЛАКОМ и 1С   | {df_diff_diagnoz.shape[0]}  | df_diff_diagnoz.xlsx |---- |---- |



''', raw=True)

 # Динамика мониторинга чистоты на 24-11-2023_08:33 :

| Параметр | Количество | Сохранен в файл |- | - |
|-:|:|:-|:-|:-|
| - __Всего__ записей в облаке| __13335__      | - | - |- |
| - возможные __опечатки__| 23                      | df_opechatka_fio_dr.xlsx | - |- |
| - __Дубли__ ФИО+ДР (с повторами)             | 6          | duble_cloude_fio.xlsx |---- |---- |
| - __Дубли__ СНИЛС (с повторами)              | 11             | df_dub_snils.xlsx |---- |---- |
| - __Дубли__ Полис-ОМС (с повторами)          | 8                  | df_dub_oms.xlsx |---- |---- |
| - пустой id-ЕМИАС (в облаке) (позже 15.11.23) | 50   | df_cloud_pusto_id_emias.xlsx |---- |---- |
| - пустой СНИЛС   (в облаке)              | 8   | df_cloud_pusto_snils.xlsx |---- |---- |
| - пустой ОМС  (в облаке)                 | 1   | df_cloud_pusto_oms.xlsx |---- |---- |
| - пустое ДР (в облаке)                 | 11   | df_cloud_pusto_dr.xlsx |---- |---- |
| - пустая дата ВК(в облаке)  (статус "не новый")              | 93   | df_cloud_pusto_vk.xlsx |---- |---- |
| - пустая МО при наличии даты ВК (пациент - не новый)               | 25 |  df_cloud_pusto_mo.xlsx |---- |---- |
| - Пустая Дата разрешения (облако) если Гражд статус "не москвич"               | 9 |  df_cloud_pusto_razresh.xlsx |---- |---- |
| - Пустой диагноз в облаке  (есть в 1с)   | 26 |  df_cloud_pusto_diagnoz.xlsx |---- |---- |
| - статус заявки "новая" продолжительность более 5 дней                | 50  | df_cloud_long_zayavka.xlsx |---- |---- |
| - статус заявки "новая", текущ статус - "не новый"                | 8  | df_cloud_status_not_new.xlsx |---- |---- |
| - прекращено оказание (в облаке) причина Пустая                | 11  | df_cloud_pusto_dr.xlsx |---- |---- |
| - прекращено оказание (в облаке) дата Пустая                | 23   |df_cloud_prekrash_data_pusto.xlsx |---- |---- |
| - причина - смерть - дата прекр Пустая                | 2   |df_cloud_dead_data_pusto.xlsx |---- |---- |
| - Отличия диагнозов между ОБЛАКОМ и 1С   | 200  | df_diff_diagnoz.xlsx |---- |---- |





In [ ]:
df_cloud_pusto_vk

In [ ]:
# --------------------------------------------

In [ ]:
# --------------------------------------------

In [ ]:
# --------------------------------------------

In [ ]:
# --------------------------------------------

In [ ]:
# | - Текущий статус пациента' облако <> 1с   | {df_differ_now_status.shape[0]}   |---- |---- |---- |

In [57]:
# **********************************

In [60]:
# df_cloud_dead_data_pusto['Дата рождения_облако'] = pd.to_datetime(df_cloud_dead_data_pusto['Дата рождения_облако']).dt.strftime('%Y-%m'))
# df_cloud_dead_data_pusto

In [ ]:
# df_cloud_dead_data_pusto.to_excel('df_cloud_dead_data_pusto.xlsx')

In [ ]:
# pd.to_datetime(df["date"].dt.strftime('%Y-%m'))